**Objetivo:** Liste `employees.name`, `departments.name` (como `department`) para cada funcionário.
- Join: employees.departmentId = departments.id

In [2]:
import pandas as pd
import duckdb as db

departments = pd.read_csv('../Dataset/Exercicios/departments.csv')

departments.head()

,id,name
0,1,Engineering
1,2,HR
2,3,Sales
3,4,Finance
4,5,Support


In [3]:
employees = pd.read_csv('../Dataset/Exercicios/employees.csv')

employees.head()

,id,name,salary,managerId,departmentId,email
0,1,Ana,9000,NaN,1,ana@corp.com
1,2,Bruno,8500,1.0,1,bruno@corp.com
2,3,Carla,8500,1.0,1,carla@corp.com
3,4,Diego,7800,2.0,1,diego@corp.com
4,5,Elisa,7200,2.0,1,elisa@corp.com


In [4]:
df = db.query(
    """
    select 
        t1.name as employee,
        t2.name as department

    from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    left join read_csv_auto('../Dataset/Exercicios/departments.csv') as t2 on t2.id = t1.departmentId
              
    """).to_df()

df

,employee,department
0,Ana,Engineering
1,Bruno,Engineering
2,Carla,Engineering
3,Diego,Engineering
4,Elisa,Engineering
5,Fabio,HR
6,Gabi,HR
7,Henrique,HR
8,Iara,Sales
9,Joao,Sales


Second Highest Salary (Medium)
**Objetivo:** Retorne o 2º maior salário distinto da tabela `employees` (coluna `salary`).
- Se não existir, retorne NULL.

In [5]:
df = db.query(
    """
    with employees as (
    select 
        name,
        dense_rank() over (order by salary desc) as rn,
        salary
    from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    )
    
    select 
        *
    from employees
    where rn = 2
             
    """).to_df()

df

,name,rn,salary
0,Nina,2,9500


---

 Nth Highest Salary (Medium)
**Objetivo:** Crie uma query (ou função, se seu SGBD permitir) que retorne o N-ésimo maior salário distinto de `employees`.
- Teste com N=1,2,3,4.

In [6]:
df = db.query(
    """
    with employees as (
        select distinct
            salary, 
            dense_rank() over(order by salary desc) as rn
        from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    )
    select * from employees order by rn
    
             
    """).to_df()

df

,salary,rn
0,11000,1
1,9500,2
2,9300,3
3,9000,4
4,8500,5
5,7800,6
6,7200,7
7,6500,8
8,6200,9
9,5900,10


---

Rank Scores (Medium)
**Tabela:** `scores(id, score)`
**Objetivo:** Retorne `score` e o `rank` denso (dense rank) em ordem decrescente.
- Mesma nota -> mesmo rank; não “pula” números.

In [7]:
df = db.query(
    """
    with scores as (
        select 
            score,
            dense_rank() over(order by score desc) as rn

        from read_csv_auto('../Dataset/Exercicios/scores.csv') as t1
    )
    select * from scores 
    
             
    """).to_df()

df

,score,rn
0,100,1
1,98,2
2,98,2
3,90,3
4,88,4
5,88,4
6,70,5
7,60,6


---

 Consecutive Numbers (Medium)
**Tabela:** `logs(id, num)`
**Objetivo:** Encontre números que aparecem **pelo menos 3 vezes consecutivas** (por id crescente).
- Retorne apenas os números.

In [8]:
df = db.query(
    """
    with scores as (
        select 
      *
        from read_csv_auto('../Dataset/Exercicios/logs.csv') as t1
    )
    select * from scores 
    
             
    """).to_df()

df

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2
7,8,2
8,9,3
9,10,3


In [ ]:
# num = prev_num = prev2_num

df = db.query(
    """
    with id as (
        select 
            id,
            num,
            lag(num) over(order by id) as prev_num,
            lag(num, 2) over(order by id) prev_num2
        from read_csv_auto('../Dataset/Exercicios/logs.csv') as t1
    )

    select 
        num
    from id 
    where   
    and num = prev_num2
    order by 1
             
    """).to_df()

df

,num
0,1
1,2


In [ ]:
# Resolvendo com autojoin

df = db.query(
    """
    select distinct 
        t1.num
    from read_csv_auto('../Dataset/Exercicios/logs.csv') as t1
    join read_csv_auto('../Dataset/Exercicios/logs.csv') as t2
        on t2.id = t1.id + 1 and t2.num = t1.num
    join read_csv_auto('../Dataset/Exercicios/logs.csv') as t3
        on t3.id = t1.id + 2 and t3.num = t1.num

    """).to_df()

df

,num
0,1
1,2


---

 Employees Earning More Than Their Managers (Easy)
**Tabela:** `employees(id, name, salary, managerId, ...)`
**Objetivo:** Retorne os nomes dos funcionários cujo `salary` é maior que o salário do seu manager.

In [21]:
df = db.query(
    """
    with employees as (
        select 
            *   
        from read_csv_auto('../Dataset/Exercicios/employees.csv') t1
    )

    select 
        *
    from employees

             
    """).to_df()

df

,id,name,salary,managerId,departmentId,email
0,1,Ana,9000,NaN,1,ana@corp.com
1,2,Bruno,8500,1.0,1,bruno@corp.com
2,3,Carla,8500,1.0,1,carla@corp.com
3,4,Diego,7800,2.0,1,diego@corp.com
4,5,Elisa,7200,2.0,1,elisa@corp.com
5,6,Fabio,5000,NaN,2,fabio@corp.com
6,7,Gabi,4800,6.0,2,gabi@corp.com
7,8,Henrique,4600,6.0,2,henrique@corp.com
8,9,Iara,6500,NaN,3,iara@corp.com
9,10,Joao,6200,9.0,3,joao@corp.com


In [30]:
df = db.query(
    """
    with employees as (
        select distinct
            t1.id,
            t1.name,
            t2.name as gerente,
            t1.managerid,
            t1.salary,
            t2.salary as salario_gerente
            
        from read_csv_auto('../Dataset/Exercicios/employees.csv') t1
         join read_csv_auto('../Dataset/Exercicios/employees.csv') t2
            on t1.managerid = t2.id and t1.salary > t2.salary 
    )

    select 
        *
    from employees


             
    """).to_df()

df

,id,name,gerente,managerId,salary,salario_gerente
0,19,Tiago,Paula,16.0,4300,4100
1,17,Rafael,Paula,16.0,4200,4100


---

Duplicate Emails (Easy)
**Tabela:** `person(id, email)`
**Objetivo:** Retorne os emails que aparecem mais de uma vez.

In [ ]:
df = db.query(
    """
    with person as (
        select  
            email,
        from read_csv_auto('../Dataset/Exercicios/person.csv') t1
        group by 1
        having count(email) > 1
    )

    select 
        email
    from person


             
    """).to_df()

df

,email
0,a@leetcode.com
1,c@leetcode.com


---

Customers Who Never Order (Easy)
**Tabelas:** `customers`, `orders`
**Objetivo:** Retorne os clientes que **não** possuem pedidos.

In [58]:
df = db.query(
    """
    with customers as (
        select  
            *
        from read_csv_auto('../Dataset/Exercicios/customers.csv') t1
        left join read_csv_auto('../Dataset/Exercicios/orders.csv') t2
            on  t1.id = t2.customerId
        where t2.id is null
    )

    select 
        name
    from customers

    """).to_df()

df

,name
0,Pet Shop Amigo
